# Get fast text
Download FastText word vectors file for Persian

In [1]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.fa.vec

In [2]:
import time
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression

# keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Input, Embedding, Dropout
from keras.layers import GlobalMaxPool1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import CuDNNLSTM, LSTM, Bidirectional
from keras.layers.convolutional import Conv1D
# from keras.utils import plot_model

# Word Embedding
from gensim.models import KeyedVectors

# Preprocessing
# from stopwords_guilannlp import stopwords_output
from hazm import *

# Visualization
import matplotlib.pyplot as plt
import shap
%matplotlib inline
pd.set_option('display.max_colwidth', None)

# Measuring metrics
from sklearn import metrics
from sklearn.metrics import f1_score

# Loading Data

In [3]:
train_df = pd.read_csv("data/train.csv", encoding="utf-8")
eval_df = pd.read_csv("data/eval.csv", encoding="utf-8")
test_df = pd.read_csv("data/test.csv", encoding="utf-8")

In [4]:
train_df.info()
print()
eval_df.info()
print()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  800 non-null    int64  
 1   comment     800 non-null    object 
 2   rate        800 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 18.9+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  200 non-null    int64  
 1   comment     200 non-null    object 
 2   rate        200 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.8+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  170 non-null    int64  
 1   comment     170 non-nul

In [5]:
# convert rate to 1 and -1 for binary classification
train_df.loc[train_df['rate'] >= 0, 'rate'] = 1
train_df.loc[train_df['rate'] < 0, 'rate'] = 0

eval_df.loc[eval_df['rate'] >= 0, 'rate'] = 1
eval_df.loc[eval_df['rate'] < 0, 'rate'] = 0

test_df.loc[test_df['rate'] >= 0, 'rate'] = 1
test_df.loc[test_df['rate'] < 0, 'rate'] = 0

In [6]:
# See the data number of sentence in each category 
from collections import Counter
cnt = Counter(train_df['rate'].values)
cnt = dict(cnt)
print(cnt)

{1.0: 606, 0.0: 194}


In [7]:
labels = list(cnt.keys())
sizes = list(cnt.values())
colors = ['#3fba36', '#66b3ff','#ffcc99','#ff9999', '#d44444']
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, colors=colors,
        autopct='%1.1f%%', startangle=90)
#draw circle
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
# Equal aspect ratio ensures that pie is drawn as a circle
ax1.axis('equal')  
plt.tight_layout()
# Decomment following line if you want to save the figure
# plt.savefig('distribution.png')
plt.show()

# Preprocess

In [8]:
puncs = ['،', '.', ',', ':', ';', '"']
normalizer = Normalizer()
lemmatizer = Lemmatizer()

# turn a doc into clean tokens
def clean_doc(doc):
    # Normalize document using Hazm Normalizer
    doc = normalizer.normalize(doc)
    # Tokenize text
    tokenized = word_tokenize(doc)
    tokens = []
    for t in tokenized:
      temp = t
      for p in puncs:
        temp = temp.replace(p, '')
      tokens.append(temp)
    # tokens = [w for w in tokens if not w in stop_set]
    # Remove stop words
    tokens = [w for w in tokens if not len(w) <= 1]
    tokens = [w for w in tokens if not w.isdigit()]
    # Lemmatize sentence words using Hazm Lemmatizer
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    tokens = ' '.join(tokens)
    return tokens

# Prepare FastText Model

In [9]:
EMBEDDING_FILE = 'wiki.fa.vec'

def import_with_gensim(file_address):
  # Creating the model
  ft_model = KeyedVectors.load_word2vec_format(file_address)
  # Getting the tokens
  ft_words = []
  for ft_word in ft_model.vocab:
      ft_words.append(ft_word)
  return ft_model, ft_words
  
ft_model, ft_words = import_with_gensim(EMBEDDING_FILE)

In [10]:
# FastText embedding dimensionality
embed_size = 300

In [11]:
# We get the mean and standard deviation of the embedding weights so that we could maintain the
# same statistics for the rest of our own random generated weights.
embedding_list = list()
for w in ft_words:
  embedding_list.append(ft_model[w])

all_embedding = np.stack(embedding_list)
emb_mean, emb_std = all_embedding.mean(), all_embedding.std()

# Apply preprocessing to data

In [12]:
# Apply preprocessing step to training data
x_train = train_df['comment'].values
y_train = train_df['rate'].values
train_docs = np.empty_like(x_train)
for index, document in enumerate(x_train):
  train_docs[index] = clean_doc(document)

In [13]:
# Apply preprocessing step to eval data
x_eval = eval_df['comment'].values
y_eval = eval_df['rate'].values
eval_docs = np.empty_like(x_eval)
for index, document in enumerate(x_eval):
  eval_docs[index] = clean_doc(document)

In [14]:
# Apply preprocessing step to eval data
x_test = test_df['comment'].values
y_test = test_df['rate'].values
test_docs = np.empty_like(x_test)
for index, document in enumerate(x_test):
  test_docs[index] = clean_doc(document)

# Setting tokenizer up

In [15]:
num_words = 2500

# Create the tokenizer
tokenizer = Tokenizer()

# fFt the tokenizer on the training documents
tokenizer.fit_on_texts(train_docs)

In [16]:
# Find maximum length of training sentences
max_length = max([len(s.split()) for s in train_docs])
print("max length:", max_length)

max length: 598


# Embed sentences

In [17]:
# Embed training sequences
encoded_docs = tokenizer.texts_to_sequences(train_docs)

# Pad embeded training sequences
x_train_padded = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [18]:
# Define vocabulary size (largest integer value)
vocab_size = len(tokenizer.word_index)
print('vocab size:', vocab_size)

vocab size: 3718


In [19]:
nb_words = len(tokenizer.word_index)

# the size will be Number of Words in Vocab X Embedding Size
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))

embeddedCount = 0
for word, i in tokenizer.word_index.items():
    i -= 1
    if word in ft_model.vocab:
        embedding_vector = ft_model[word]
        embedding_matrix[i] = embedding_vector
        embeddedCount += 1
    # Unknown words
    else:  
        embedding_vector = ft_model['subdivision_name']
        embedding_matrix[i] = embedding_vector
        embeddedCount += 1

print('total embedded:', embeddedCount, 'common words')
print('Embedding matrix shape:', embedding_matrix.shape)

total embedded: 3718 common words
Embedding matrix shape: (3718, 300)


In [20]:
# Embed testing sequences
encoded_docs = tokenizer.texts_to_sequences(test_docs)
# Pad testing sequences
x_test_padded = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [21]:
# Embed eval sequences
encoded_docs = tokenizer.texts_to_sequences(eval_docs)
# Pad eval sequences
x_eval_padded = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

# Model

# CNN Model

In [22]:
model_cnn = Sequential()
model_cnn.add(Embedding(vocab_size, embedding_matrix.shape[1], weights=[embedding_matrix], trainable=False))
model_cnn.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same'))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Conv1D(filters=64, kernel_size=7, activation='relu', padding='same'))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dropout(0.1))
model_cnn.add(Dense(500, activation="sigmoid"))
model_cnn.add(Dense(1, activation='sigmoid'))

In [23]:
model_cnn.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_cnn.summary()
batch_size_cnn = 64
epochs_cnn = 10

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 300)         1115400   
                                                                 
 conv1d (Conv1D)             (None, None, 64)          57664     
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 64)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, None, 64)          20544     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, None, 64)         0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, None, 64)          2

In [24]:
# Train model
hist_cnn = model_cnn.fit(x_train_padded, y_train, batch_size=batch_size_cnn, epochs=epochs_cnn)

Epoch 1/10
13/13 [==============================] - 2s 31ms/step - loss: 0.6813 - accuracy: 0.6375
Epoch 2/10
13/13 [==============================] - 0s 10ms/step - loss: 0.5515 - accuracy: 0.7575
Epoch 3/10
13/13 [==============================] - 0s 9ms/step - loss: 0.5492 - accuracy: 0.7575
Epoch 4/10
13/13 [==============================] - 0s 9ms/step - loss: 0.5435 - accuracy: 0.7575
Epoch 5/10
13/13 [==============================] - 0s 9ms/step - loss: 0.5352 - accuracy: 0.7575
Epoch 6/10
13/13 [==============================] - 0s 9ms/step - loss: 0.5206 - accuracy: 0.7575
Epoch 7/10
13/13 [==============================] - 0s 9ms/step - loss: 0.4869 - accuracy: 0.7575
Epoch 8/10
13/13 [==============================] - 0s 9ms/step - loss: 0.4412 - accuracy: 0.7875
Epoch 9/10
13/13 [==============================] - 0s 10ms/step - loss: 0.3781 - accuracy: 0.8275
Epoch 10/10
13/13 [==============================] - 0s 9ms/step - loss: 0.2639 - accuracy: 0.9150


In [25]:
def eval_model(model):
    loss, acc = model.evaluate(x_train_padded, y_train, verbose=0)
    print('Train Accuracy: %f' % (acc))
    loss, acc = model.evaluate(x_eval_padded, y_eval, verbose=0)
    print('Eval Accuracy: %f' % (acc))
    loss, acc = model.evaluate(x_test_padded, y_test, verbose=0)
    print('Test Accuracy: %f' % (acc))


In [26]:
# Evaluate model
eval_model(model_cnn)

Train Accuracy: 0.946250
Eval Accuracy: 0.710000
Test Accuracy: 0.705882


# B-LSTM Model

In [27]:
model_blstm = Sequential()
model_blstm.add(Embedding(vocab_size, embedding_matrix.shape[1], weights=[embedding_matrix], trainable=False))
model_blstm.add(Bidirectional(CuDNNLSTM(300, return_sequences=True, name='lstm_layer')))
model_blstm.add(GlobalMaxPool1D())
model_blstm.add(Dropout(0.1))
model_blstm.add(Dense(300, activation="relu"))
model_blstm.add(Dropout(0.1))
model_blstm.add(Dense(1, activation='sigmoid'))

In [28]:
model_blstm.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_blstm.summary()
batch_size_blstm = 32
epochs_blstm = 10

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 300)         1115400   
                                                                 
 bidirectional (Bidirectiona  (None, None, 600)        1444800   
 l)                                                              
                                                                 
 global_max_pooling1d_1 (Glo  (None, 600)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 600)               0         
                                                                 
 dense_2 (Dense)             (None, 300)               180300    
                                                                 
 dropout_2 (Dropout)         (None, 300)              

In [29]:
# Train model
hist_blstm = model_blstm.fit(x_train_padded, y_train, batch_size=batch_size_blstm, epochs=epochs_blstm)

Epoch 1/10
25/25 [==============================] - 4s 109ms/step - loss: 0.5800 - accuracy: 0.7575
Epoch 2/10
25/25 [==============================] - 3s 109ms/step - loss: 0.5522 - accuracy: 0.7575
Epoch 3/10
25/25 [==============================] - 3s 109ms/step - loss: 0.5312 - accuracy: 0.7575
Epoch 4/10
25/25 [==============================] - 3s 109ms/step - loss: 0.4953 - accuracy: 0.7588
Epoch 5/10
25/25 [==============================] - 3s 109ms/step - loss: 0.4643 - accuracy: 0.7800
Epoch 6/10
25/25 [==============================] - 3s 109ms/step - loss: 0.3820 - accuracy: 0.8238
Epoch 7/10
25/25 [==============================] - 3s 109ms/step - loss: 0.2454 - accuracy: 0.9137
Epoch 8/10
25/25 [==============================] - 3s 109ms/step - loss: 0.1514 - accuracy: 0.9525
Epoch 9/10
25/25 [==============================] - 3s 109ms/step - loss: 0.0779 - accuracy: 0.9812
Epoch 10/10
25/25 [==============================] - 3s 109ms/step - loss: 0.0392 - accuracy: 0.9925

In [30]:
# Evaluate model
eval_model(model_blstm)

Train Accuracy: 0.997500
Eval Accuracy: 0.715000
Test Accuracy: 0.711765
